In [ ]:
'''
here we will be analysing the horses health using the decison tree , random forest tree and
compare the accuracy matrix of the both models
For Descion Tress and random Forest we dont need to do standadisation 
that is checking outliers, distribution, co-leaniarty is there or not
'''

In [32]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score,f1_score,recall_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')


In [6]:
horseDataFrame = pd.read_csv('/Users/uday_kumar_swamy/Library/CloudStorage/OneDrive-Personal/datascience_videos/machinelearningpractice/machinelearning/dataset/horse.csv')

In [7]:
#lets check the shape of the dataframe
horseDataFrame.shape

(299, 28)

In [8]:
#now lets check the inforation about the dataframe
horseDataFrame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   surgery                299 non-null    object 
 1   age                    299 non-null    object 
 2   hospital_number        299 non-null    int64  
 3   rectal_temp            239 non-null    float64
 4   pulse                  275 non-null    float64
 5   respiratory_rate       241 non-null    float64
 6   temp_of_extremities    243 non-null    object 
 7   peripheral_pulse       230 non-null    object 
 8   mucous_membrane        252 non-null    object 
 9   capillary_refill_time  267 non-null    object 
 10  pain                   244 non-null    object 
 11  peristalsis            255 non-null    object 
 12  abdominal_distention   243 non-null    object 
 13  nasogastric_tube       195 non-null    object 
 14  nasogastric_reflux     193 non-null    object 
 15  nasoga

In [14]:
#now lets check the outcome unique values for the imbalence
#data seems evenly distributed
#normalize=True --> will give the percentage instade of the absolute value
#it is multi-class classification --> have multiple target variables
horseDataFrame.outcome.value_counts(normalize=True)

lived         0.595318
died          0.257525
euthanized    0.147157
Name: outcome, dtype: float64

In [10]:
#now lets view the data and see possible unwanted columns for the analysis and drop them
horseDataFrame.head(5)

,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,...,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,outcome,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data
0,no,adult,530101,38.5,66.0,28.0,cool,reduced,NaN,more_3_sec,...,45.0,8.4,NaN,NaN,died,no,11300,0,0,no
1,yes,adult,534817,39.2,88.0,20.0,NaN,NaN,pale_cyanotic,less_3_sec,...,50.0,85.0,cloudy,2.0,euthanized,no,2208,0,0,no
2,no,adult,530334,38.3,40.0,24.0,normal,normal,pale_pink,less_3_sec,...,33.0,6.7,NaN,NaN,lived,no,0,0,0,yes
3,yes,young,5290409,39.1,164.0,84.0,cold,normal,dark_cyanotic,more_3_sec,...,48.0,7.2,serosanguious,5.3,died,yes,2208,0,0,yes
4,no,adult,530255,37.3,104.0,35.0,NaN,NaN,dark_cyanotic,more_3_sec,...,74.0,7.4,NaN,NaN,died,no,4300,0,0,no


In [11]:
#clearly we dont want the hospital number for our analysis so will proceed to drop it
horseDataFrame.drop(columns=['hospital_number'],axis=1,inplace=True)

In [12]:
#now lets check for the missing values in the dataframe
horseDataFrame.isna().sum()

surgery                    0
age                        0
rectal_temp               60
pulse                     24
respiratory_rate          58
temp_of_extremities       56
peripheral_pulse          69
mucous_membrane           47
capillary_refill_time     32
pain                      55
peristalsis               44
abdominal_distention      56
nasogastric_tube         104
nasogastric_reflux       106
nasogastric_reflux_ph    246
rectal_exam_feces        102
abdomen                  118
packed_cell_volume        29
total_protein             33
abdomo_appearance        165
abdomo_protein           198
outcome                    0
surgical_lesion            0
lesion_1                   0
lesion_2                   0
lesion_3                   0
cp_data                    0
dtype: int64

In [24]:
#here we will try to fill the missing value's with most-frequent and mean using the imputer

numericImputer = SimpleImputer(strategy='mean')
categoricalImputer = SimpleImputer(strategy='most_frequent')

numeric_features = horseDataFrame.select_dtypes(include=['int64','float64']).columns
categorical_features = horseDataFrame.select_dtypes(include=['object']).columns
preprocessing = ColumnTransformer( 
    transformers = [
    ('num',numericImputer,numeric_features),
    ('cat',categoricalImputer,categorical_features) 
],
 remainder='passthrough'  # To keep columns that are not explicitly selected
)

horseDataframe_transformed = preprocessing.fit_transform(horseDataFrame)

horseDataframe_transformed = pd.DataFrame(horseDataframe_transformed,columns=numeric_features.tolist()+categorical_features.tolist())


for col in numeric_features:
    horseDataframe_transformed[col] = horseDataframe_transformed[col].astype(horseDataFrame[col].dtype)

for col in categorical_features:
    horseDataframe_transformed[col] = horseDataframe_transformed[col].astype(horseDataFrame[col].dtype)
    


In [25]:
#validate the missing values
horseDataframe_transformed.isna().sum()

rectal_temp              0
pulse                    0
respiratory_rate         0
nasogastric_reflux_ph    0
packed_cell_volume       0
total_protein            0
abdomo_protein           0
lesion_1                 0
lesion_2                 0
lesion_3                 0
surgery                  0
age                      0
temp_of_extremities      0
peripheral_pulse         0
mucous_membrane          0
capillary_refill_time    0
pain                     0
peristalsis              0
abdominal_distention     0
nasogastric_tube         0
nasogastric_reflux       0
rectal_exam_feces        0
abdomen                  0
abdomo_appearance        0
outcome                  0
surgical_lesion          0
cp_data                  0
dtype: int64

In [26]:
#now will do lable encoding
categorical_features.tolist()

['surgery',
 'age',
 'temp_of_extremities',
 'peripheral_pulse',
 'mucous_membrane',
 'capillary_refill_time',
 'pain',
 'peristalsis',
 'abdominal_distention',
 'nasogastric_tube',
 'nasogastric_reflux',
 'rectal_exam_feces',
 'abdomen',
 'abdomo_appearance',
 'outcome',
 'surgical_lesion',
 'cp_data']

In [28]:
le = LabelEncoder()
'''
here we are encoing all the object type of the data
into labelencoded data
'''
for col in categorical_features:
    horseDataframe_transformed[col] = le.fit_transform(horseDataframe_transformed[col])

In [30]:
#check the info of the data now post encoding
horseDataframe_transformed.info()
#now the data is ready for the machine learning process as all the columns are numeric

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   rectal_temp            299 non-null    float64
 1   pulse                  299 non-null    float64
 2   respiratory_rate       299 non-null    float64
 3   nasogastric_reflux_ph  299 non-null    float64
 4   packed_cell_volume     299 non-null    float64
 5   total_protein          299 non-null    float64
 6   abdomo_protein         299 non-null    float64
 7   lesion_1               299 non-null    int64  
 8   lesion_2               299 non-null    int64  
 9   lesion_3               299 non-null    int64  
 10  surgery                299 non-null    int64  
 11  age                    299 non-null    int64  
 12  temp_of_extremities    299 non-null    int64  
 13  peripheral_pulse       299 non-null    int64  
 14  mucous_membrane        299 non-null    int64  
 15  capill

In [31]:
'''
now lets split the data into target and feature
'''
feature = horseDataframe_transformed.drop(columns=['outcome'],axis = 1)
target = horseDataframe_transformed['outcome']

In [39]:
'''
now lets split the data into test and train using sklearn model selection
'''
X_train,X_test,y_train,y_test = train_test_split(feature, target, train_size=0.7,random_state=4)


In [42]:
'''
now lets train DecisionTreeClassifier , here im selecting the criteria as entropy 
but default will be gini index
'''
desicion_tree = DecisionTreeClassifier(criterion='entropy')
desicion_tree.fit(X_train,y_train)
y_pred_train = desicion_tree.predict(X_train)
y_pred_test = desicion_tree.predict(X_test)

In [43]:
train_accuracy = accuracy_score(y_train,y_pred_train)
test_accuracy = accuracy_score(y_test,y_pred_test)
train_accuracy,test_accuracy

(1.0, 0.6555555555555556)

In [79]:
'''
now lets train RandomForestClassifier , here im selecting the criteria as entropy 
but default will be gini index
'''
randomForest = RandomForestClassifier(criterion='entropy',max_depth=2,random_state=2,min_samples_split=15)
randomForest.fit(X_train,y_train)
y_pred_train = randomForest.predict(X_train)
y_pred_test = randomForest.predict(X_test)

train_accuracy = accuracy_score(y_train,y_pred_train) 
test_accuracy = accuracy_score(y_test,y_pred_test)
train_accuracy,test_accuracy

(0.69377990430622, 0.6222222222222222)

In [70]:
'''
gives the importance of the feature in descion making
'''
randomForest.feature_importances_

array([0.01311703, 0.07911075, 0.0078115 , 0.00696117, 0.18057496,
       0.08561815, 0.07612776, 0.17978619, 0.        , 0.        ,
       0.02959042, 0.00243512, 0.03042027, 0.01776197, 0.01582552,
       0.06197802, 0.03738274, 0.00127196, 0.01759563, 0.00502481,
       0.03773061, 0.01431153, 0.00756671, 0.01977237, 0.0681438 ,
       0.00408102])

In [75]:
X_train.columns.sort_values(ascending=False)

Index(['total_protein', 'temp_of_extremities', 'surgical_lesion', 'surgery',
       'respiratory_rate', 'rectal_temp', 'rectal_exam_feces', 'pulse',
       'peristalsis', 'peripheral_pulse', 'pain', 'packed_cell_volume',
       'nasogastric_tube', 'nasogastric_reflux_ph', 'nasogastric_reflux',
       'mucous_membrane', 'lesion_3', 'lesion_2', 'lesion_1', 'cp_data',
       'capillary_refill_time', 'age', 'abdomo_protein', 'abdomo_appearance',
       'abdominal_distention', 'abdomen'],
      dtype='object')